## <center> Специализация "Машинное обучение и анализ данных"
# <center> Проект "Алиса":<br>Идентификация пользователей по посещенным веб-страницам
## <div style="text-align: right">Автор: Пётр Зюзиков</div>

### Исходные данные:
В нашем распоряжении есть база данных пользователей и их рабочих сессий в Интернете: последовательностей из нескольких веб-сайтов, посещенных подряд. Данные взяты из [соревнования "Catch Me If You Can"](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2).
### Задача:
Найти наиболее подходящий алгоритм и обучить его на этих данных, чтобы максимально успешно определять, принадлежит ли сессия определённому пользователю (бинарная классификация). Решаем для пользователя Алисы (Alice).
**Метрика: ROC AUC**
### Определение успеха:
1) Написать стабильно работающий код модели.<br>
2) Улучшить метрику в сравнении с эталонным решением.<br>
Личная цель: Войти в топ 5% лучших решений среди 4000+ студентов за всё время или максимально подойти к этому рубежу.

# <center>Промежуточные итоги

### ROC AUC
**Public: 0.95154, cross-validation: 0.00000. Эталон: 0.94928, 0.92647.**

<center>
<img src="https://i.imgur.com/bcA7Cvx.png"/>
</center>

### Особенности решения:
Начну с вещи, которую сознательно не стал делать: leaderboard probing. Из-за текущего формата соревнования доступен только публичный лидерборд и нет возможности узнать итоговые результаты на приватной части тестов. И это открывает широкие возможности для махинаций с файлом ответов. Вместо этого реализована кросс-валидация с правильной зависимостью от времени, и она настолько же важна, как и результаты публичного лидерборда.

В эталонном решении значение кросс-валидации и наиболее подходящий гиперпараметр неверны из-за ошибки формата даты в записи данных, что хорошо показано в kaggle notebook [Model validation in a competition - Fixing CV](https://www.kaggle.com/sgdread/model-validation-in-a-competition-fixing-cv). Я добавил исправление этой ошибки в эталонное решение.

Исследование на Data Leaks похоже на EDA, поэтому его я с удовольствием провёл. Тренировочные данные даны нам в двух форматах, тестовые в одном. И благодаря пониманию логики преобразования тренировочных данных можно объединить часть тестовых, и выдать для этих объединений более точные прогнозы. Здесь особенно пригодилась библиотека numpy, так как с pandas обработка данных занимала слишком много времени и усложняла тестирование.

Добавим в модель новые признаки: особенно нам интересны признаки, которые объединяют время посещения и название сайта. В эталонном решении уже хорошо отобраны признаки, которые учитывают эти показатели независимо.

### <center>Шаблон решения
Этот исходный код — [от организаторов](https://www.kaggle.com/kashnitsky/alice-template-for-your-solution), с добавленным мной кодом исправления ошибки в данных. Задания курса в течение предыдущих 6 недель включают в себя самостоятельную реализацию кода всех частей этого решения. Ноутбуки c моими решениями можно посмотреть в папке Preparation.

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import time
from contextlib import contextmanager
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
PATH_TO_DATA = 'kaggle'
AUTHOR = 'EyeShield77'
SEED = 17
N_JOBS = 4
NUM_TIME_SPLITS = 10    # for time-based cross-validation
SITE_NGRAMS = (1, 5)    # site ngrams for "bag of sites"
MAX_FEATURES = 50000    # max features for "bag of sites"
BEST_LOGIT_C = 5.45559  # precomputed tuned C for logistic regression

TIMES = ['time%s' % i for i in range(1, 11)]
SITES = ['site%s' % i for i in range(1, 11)]

# reporting running times
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

# fixing data parsing mistake
def fix_incorrect_date_formats(df, columns_to_fix):
    for time in columns_to_fix:
        d = df[time]
        d_fix = d[d.dt.day <= 12]
        d_fix = pd.to_datetime(d_fix.apply(str), format='%Y-%d-%m %H:%M:%S')
        df.loc[d_fix.index.values, time] = d_fix
    return df

def prepare_sparse_features(after_load_fn, path_to_train, path_to_test, path_to_site_dict, vectorizer_params):
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=TIMES)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=TIMES)

    # Fixing the dates
    train_df = after_load_fn(train_df)
    test_df = after_load_fn(test_df)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    train_sessions = train_df[SITES].fillna(0).astype('int').apply(lambda row:
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[SITES].fillna(0).astype('int').apply(lambda row:
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[TIMES], test_df[TIMES]
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times


def add_features(times, X_sparse):
    hour = times['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <= 6)).astype('int').values.reshape(-1, 1)
    sess_duration = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[s]')\
		   .astype('int').values.reshape(-1, 1)
    day_of_week = times['time1'].apply(lambda t: t.weekday()).values.reshape(-1, 1)
    month = times['time1'].apply(lambda t: t.month).values.reshape(-1, 1) 
    year_month = times['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5

    X = hstack([X_sparse, morning, day, evening, night, sess_duration, day_of_week, month, year_month])
    return X

with timer('Building sparse site features'):
    X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times = \
        prepare_sparse_features(
            after_load_fn=(lambda df: fix_incorrect_date_formats(df, TIMES)), # Applying dates fix
            path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
            path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
            path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
            vectorizer_params={'ngram_range': SITE_NGRAMS,
                               'max_features': MAX_FEATURES,
                               'tokenizer': lambda s: s.split()})

with timer('Building additional features'):
    X_train_final = add_features(train_times, X_train_sites)
    X_test_final = add_features(test_times, X_test_sites)


with timer('Cross-validation'):
    time_split = TimeSeriesSplit(n_splits=NUM_TIME_SPLITS)
    logit = LogisticRegression(random_state=SEED, solver='liblinear')

    c_values = [BEST_LOGIT_C]

    logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=N_JOBS, cv=time_split, verbose=1)
    logit_grid_searcher.fit(X_train_final, y_train)
    print('CV score', logit_grid_searcher.best_score_)


with timer('Test prediction and submission'):
    test_pred = logit_grid_searcher.predict_proba(X_test_final)[:, 1]
    pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
    pred_df.to_csv(f'submission_alice_{AUTHOR}.csv', index_label='session_id')

[Building sparse site features] done in 35 s
[Building additional features] done in 6 s
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   17.2s finished


CV score 0.926474775872798
[Cross-validation] done in 26 s
[Test prediction and submission] done in 0 s


Проведём подбор гиперпараметров: очень вероятно, что лучший параметр, подобранный организаторами, не подходит для исправленного датасета.

In [3]:
with timer('Cross-validation'):
    time_split = TimeSeriesSplit(n_splits=NUM_TIME_SPLITS)
    logit = LogisticRegression(random_state=SEED, solver='liblinear')

    #c_values = np.logspace(-2, 2, 20)
    #c_values = np.linspae(2, 6, 41)
    c_values = [3]
    
    logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=N_JOBS, cv=time_split, verbose=1)
    logit_grid_searcher.fit(X_train_final, y_train)
    print('CV score:', logit_grid_searcher.best_score_)
    print('Best logit C:', logit_grid_searcher.best_params_)


with timer('Test prediction and submission'):
    test_pred = logit_grid_searcher.predict_proba(X_test_final)[:, 1]
    pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
    pred_df.to_csv(f'submission_alice_{AUTHOR}2.csv', index_label='session_id')

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   13.6s finished


CV score: 0.9269516972229194
Best logit C: {'C': 3}
[Cross-validation] done in 22 s
[Test prediction and submission] done in 0 s


Итак, наш новый ориентир: 0.94998 (СV — 0.92695).

## EDA: Исследуем данные, придумываем новые признаки

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки) и посмотрим на них.**

In [4]:
PATH_TO_DATA = 'kaggle/'
TIMES = ['time%d' % i for i in range(1, 11)]
SITES = ['site%d' % i for i in range(1, 11)]
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id', parse_dates=TIMES)
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id', parse_dates=TIMES)
print('train_df.shape =', train_df.shape)
print('test_df.shape =', test_df.shape)
print('Разделение на классы в тренировочной выборке:')
print('Элис:', train_df['target'].value_counts()[1], ', остальные:', train_df['target'].value_counts()[0])

train_df.shape = (253561, 21)
test_df.shape = (82797, 20)
Разделение на классы в тренировочной выборке:
Элис: 2297 , остальные: 251264


In [5]:
train_df.head(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0


In [6]:
test_df.head(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36


В выборках мы видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут.

**Настоящие исходные данные.** Но это не все данные, которые нам доступны. Есть ещё архив train.zip с сессиями каждого пользователя в формате timestamp & site: время и посещённый сайт. И именно эти данные предобработали в тренировочную выборку, и по такому же алгоритму тестовые данные обработали в тестовую.
Сессии делили по полные по 10 сайтов до момента, пока не появлялся пропуск более получаса. На этом моменте записывали "короткую" сессию из 1-10 сайтов и NaN на остальных местах.

**Также давайте вспомним, как люди в 2013-2014 году выходят в Интернет.** Если кратко, то с компьютера ровно так же, как и сейчас: запускается браузер с окнами, которые остались с предыдущей сессии. В примере снизу на примере данных Элис мы видим, что за 5 секунд она открыла 9 сайтов. Есть и альтернативная гипотеза: приличная часть обращений не является открытой новой вкладкой в браузере, это переадресации или другие автоматические действия браузера/открытых сайтов.

In [7]:
alice_df = pd.read_csv('kaggle/Alice_log.csv', parse_dates=['timestamp'])
print(f'Количество записей: {alice_df.shape[0]}')
alice_df[4:13]

Количество записей: 22769


,timestamp,site
4,2013-02-12 16:32:24,www.google.fr
5,2013-02-12 16:32:25,www.info-jeunes.net
6,2013-02-12 16:32:25,www.google.fr
7,2013-02-12 16:32:26,www.info-jeunes.net
8,2013-02-12 16:32:27,platform.twitter.com
9,2013-02-12 16:32:27,www.info-jeunes.net
10,2013-02-12 16:32:27,www.facebook.com
11,2013-02-12 16:32:28,www.info-jeunes.net
12,2013-02-12 16:32:29,twitter.com


Посмотрим, сколько на самом деле было получасовых промежутков между открытиями сайтов у Элис. А также 20, 15, 10, 5, 3, 2 и 1-минутных.

In [8]:
alice_diff = (np.diff(alice_df['timestamp'])/np.timedelta64(1, 's'))
for i in [30, 20, 15, 10, 5, 3, 2, 1]:
    print(f'{i:2.0f} min: {len(alice_diff[alice_diff > 60*i])}')

30 min: 34
20 min: 39
15 min: 48
10 min: 62
 5 min: 100
 3 min: 155
 2 min: 220
 1 min: 406


Какое количество сайтов в каждой сессии Алисы, которые она успевает посмотреть до 5-минутного перерыва?

In [9]:
intervals = np.diff(np.where(alice_diff > 60*5))[0]
intervals

array([ 553,    2,   46,    3,  372, 3691,   71,    2,  100,  113,  133,
        297,  211,    2,  153,    5,  101,   75,   65,  759,   38,   14,
        945,   10, 1417,    1, 1046,    1,    4,    1,   96,  179,  349,
         56,    2,   87,   21,    6,   23,    1,  286,   48,   66,   21,
        108,    2,   56,   25,    1,   95,    2,  732,   51,  733,   39,
          1,  455,  124,    5,  648,  113,    2,   23,   46,    1,   99,
        162,   28,   14,    1,  442,   99,    1,   36,    1,  191,    1,
         52,   35, 2692,    3,  219,  278,  469,  212,  117,    5,   28,
        121,   99,  955,   17,  725,  450,    1,    4,    2,  457,    3],
      dtype=int64)

Очевидно, что когда мы видим в ответе на предыдущий вопрос числа типа 3691 или любые другие больше нескольких сотен, то должны понимать, что на приличную часть сайтов Алиса не заходит сознательно, это автоматический процесс. Посмотрим, какие сайты и насколько часто открываются первыми после перерыва хотя бы в 20 секунд: скорее всего, эти сайты Алиса открывает осознанно.

In [10]:
SECONDS_BETWEEN_SESSIONS = 20

intervals = np.diff(np.where(alice_diff >= SECONDS_BETWEEN_SESSIONS))[0]
site_freq = []
idx = 2
for i, td in enumerate(intervals):
    idx += td
    site_freq.append(alice_df['site'].iloc[idx])
uniq, counts = np.unique(site_freq, return_counts=True)
path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl')
with open(path_to_site_dict, 'rb') as f:
    site2id = pickle.load(f)
sites_of_interest = sorted([[counts[i], site2id[uniq[i]], uniq[i]] for i in np.where(counts >= 13)[0]], reverse=True)
print('Count, id, site name')
sites_of_interest

Count, id, site name


[[158, 21, 'www.google.fr'],
 [58, 80, 's.youtube.com'],
 [53, 37, 'twitter.com'],
 [50, 3000, 'vk.com'],
 [40, 77, 'i1.ytimg.com'],
 [39, 270, 'api.bing.com'],
 [38, 335, 'www.dailymotion.com'],
 [33, 733, 'translate.google.fr'],
 [33, 76, 'www.youtube.com'],
 [30, 677, 'drive.google.com'],
 [30, 81, 'r4---sn-gxo5uxg-jqbe.googlevideo.com'],
 [25, 616, 'docs.google.com'],
 [22, 27221, 'apiv1.scribblelive.com'],
 [17, 1919, 'syndication.twitter.com'],
 [17, 29, 'www.facebook.com'],
 [16, 879, 'r1---sn-gxo5uxg-jqbe.googlevideo.com'],
 [16, 82, 'r2---sn-gxo5uxg-jqbe.googlevideo.com'],
 [16, 22, 'apis.google.com'],
 [14, 17283, 'www.demotivateur.fr'],
 [14, 5302, 'login.vk.com'],
 [14, 229, 'clients1.google.fr'],
 [14, 63, 'ieonline.microsoft.com'],
 [13, 7832, 'www.info-jeunes.net']]

**Итак, это будет одним из наших признаков: есть ли перерыв, и является ли открытый сайт после этого перерыва одним из сайтов выше.**

Есть ли такая же закономерность в сайтах, которыми Алиса заканчивает сессию? Если на сайте заканчивается сессия, это может означать, что Алиса осталась на нём сидеть.

In [11]:
SECONDS_BETWEEN_SESSIONS = 120

intervals = np.diff(np.where(alice_diff > SECONDS_BETWEEN_SESSIONS))[0]
site_freq = []
idx = 2
for td in intervals:
    idx += td
#    site_freq.append('.'.join(alice_df['site'].iloc[idx-1].split('.')[-2:]))
    site_freq.append(alice_df['site'].iloc[idx-1])
uniq, counts = np.unique(site_freq, return_counts=True)
sites_of_interest = sorted([[counts[i], site2id[uniq[i]], uniq[i]] for i in np.where(counts >= 10)[0]], reverse=True)
print('Count, id, site name')
sites_of_interest

Count, id, site name


[[46, 3000, 'vk.com'],
 [26, 21, 'www.google.fr'],
 [19, 677, 'drive.google.com'],
 [10, 63, 'ieonline.microsoft.com']]

В обоих группах значительно выделяются vk.com и сервисы google.

**Нам интересно добавление признаков, сочетающих комбинацию условий по времени и имени сайтов: в эталонном решении таких признаков нет.** Там используются признаки отдельно по времени и отдельно по комбинации открываемых сайтов.

Чем больше id сайта, тем реже на него заходили другие пользователи. Наиболее интересными для нас являются комбинации не очень часто посещаемых сайтов общей популяцией, но на которые достаточно часто заходит Алиса. Посещения остальных сайтов отловят признаки TfidfVectorizer.

# Собираем итоговое решение

### Обновлённая функция подготовки данных

In [12]:
TIMES = ['time%s' % i for i in range(1, 11)]
SITES = ['site%s' % i for i in range(1, 11)]

def prepare_sparse_features(after_load_fn, path_to_train, path_to_test, path_to_site_dict, vectorizer_params):
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=TIMES)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=TIMES)

    # Fixing the dates
    train_df = after_load_fn(train_df)
    test_df = after_load_fn(test_df)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[TIMES], test_df[TIMES]
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    train_sessions = train_df[SITES].fillna(0).astype('int').apply(lambda row:
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[SITES].fillna(0).astype('int').apply(lambda row:
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    
    #Combining short sessions with full ones
    test_df = test_df.sort_values(by='time1')
    indices = test_df.index.tolist()
    ttest = np.array(test_df[TIMES])
    rows_to_skip = []
    row_value_to_insert = []
    res = 0
    for i, t1 in enumerate(ttest[1:, 0], start=1):
        if np.isnat(ttest[i, 9]):
            for j in range(1, min(i+1, 11)):
                if i-j not in rows_to_skip and 0 <= (t1-ttest[i-j, 9])/np.timedelta64(1, 's') <= 2:
                    res += 1
                    test_sessions[indices[i-j]-1] += ' ' \
                            + ' '.join([word for word in test_sessions[indices[i]-1].split() if word != 'unknown'])
                    test_sessions[indices[i]-1] = test_sessions[indices[i-j]-1]
                    rows_to_skip.append(indices[i]-1)
                    row_value_to_insert.append(indices[i-j]-1)
                    break
    #print('test =', res)
    
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need sessions with site ids for future engineering
    train_sessions_ids = train_df[SITES].fillna(0).astype('int').apply(lambda row: list(row), axis=1).tolist()
    test_sessions_ids = test_df[SITES].fillna(0).astype('int').apply(lambda row: list(row), axis=1).tolist()
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times, train_sessions_ids, test_sessions_ids, \
            rows_to_skip, row_value_to_insert

In [13]:
with timer('Building sparse site features'):
    X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times, train_sessions, test_sessions, rows1, rows2 = \
        prepare_sparse_features(
            after_load_fn=(lambda df: fix_incorrect_date_formats(df, TIMES)), # Applying dates fix
            path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
            path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
            path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
            vectorizer_params={'ngram_range': SITE_NGRAMS,
                               'max_features': MAX_FEATURES,
                               'tokenizer': lambda s: s.split()})

[Building sparse site features] done in 60 s


### Подготовка структур для создания новых признаков

In [14]:
SECONDS_BETWEEN_SESSIONS = 20

intervals = np.diff(np.where(alice_diff >= SECONDS_BETWEEN_SESSIONS))[0]
site_freq, site_freq2 = [], []
idx = 2
for i, td in enumerate(intervals):
    idx += td
    site_freq.append(alice_df['site'].iloc[idx])
    if i < len(intervals)-1 and intervals[i+1] > 1:
        site_freq2.append(alice_df['site'].iloc[idx+1])
uniq, counts = np.unique(site_freq, return_counts=True)
path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl')
with open(path_to_site_dict, 'rb') as f:
    site2id = pickle.load(f)
id2site = {v:k for (k, v) in site2id.items()}
uniq = [site2id[site] for site in uniq]
sites_of_interest = [uniq[i] for i in np.where(counts >= 10)[0] if uniq[i] >= 500]
alice_sites = set([site2id[site] for site in alice_df['site']])
uniq = set(uniq).union(set([site2id[site] for site in site_freq2]))

### Обновлённая функция по добавлению новых признаков

In [15]:
def add_features(times, sessions, X_sparse, sites_of_interest, uniq, alice_sites):
    hour = times['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <= 6)).astype('int').values.reshape(-1, 1)
#     sess_duration = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[s]')\
# 		   .astype('int').values.reshape(-1, 1)
    day_of_week = times['time1'].apply(lambda t: t.weekday()).values.reshape(-1, 1)
    month = times['time1'].apply(lambda t: t.month).values.reshape(-1, 1) 
    year_month = times['time1'].apply(lambda t: 100 * t.year + t.month).values.reshape(-1, 1) / 1e5
    
    times = np.array(times)
    new_features = np.zeros((len(times), len(sites_of_interest)+1), int)
    for i, session in enumerate(sessions):
        for j, tdiff in enumerate(np.diff(times[i, :]), start=1):
            if tdiff/np.timedelta64(1, 's') >= SECONDS_BETWEEN_SESSIONS:
                if session[j] not in uniq and session[j] in alice_sites:
                    new_features[i, -1] = 1
                if session[j] in sites_of_interest:
                    new_features[i, sites_of_interest.index(session[j])] = 1
    X = hstack([X_sparse, morning, day, evening, night, day_of_week, month, year_month, new_features])
#     X = hstack([X_sparse, morning, day, evening, night, day_of_week, month, year_month])
    return X

In [16]:
with timer('Building additional features'):
    X_train_final = add_features(train_times, train_sessions, X_train_sites, sites_of_interest, uniq, alice_sites)
    X_test_final = add_features(test_times, test_sessions, X_test_sites, sites_of_interest, uniq, alice_sites)

[Building additional features] done in 18 s


### Рассчёт кросс-валидации и подбор гиперпараметра

In [ ]:
with timer('Cross-validation'):
    time_split = TimeSeriesSplit(n_splits=NUM_TIME_SPLITS)
    logit = LogisticRegression(random_state=SEED, solver='liblinear')

    c_values = np.linspace(2, 4, 21)

    logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=N_JOBS, cv=time_split, verbose=1)
    logit_grid_searcher.fit(X_train_final, y_train)
    print('CV score', logit_grid_searcher.best_score_)
    print('Best logit C:', logit_grid_searcher.best_params_)

Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   30.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 210 out of 210 | elapsed:  2.9min finished


### Создание submission файла и его правка

In [ ]:
with timer('Test prediction and submission'):
    test_pred = logit_grid_searcher.predict_proba(X_test_final)[:, 1]
    pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
    pred_df.to_csv(f'submission_alice_{AUTHOR}2.csv', index_label='session_id')

In [ ]:
answers = pd.read_csv('submission_alice_EyeShield77_2.csv', header=0, index_col='session_id')
ans = np.array(answers['target'])
for i, j in zip(rows1, rows2):
    answers['target'].iloc[i] = answers['target'].iloc[j]
answers.to_csv(f'submission_alice_EyeShield77_upd', index_label='session_id')